<a href="https://colab.research.google.com/github/TiaSinghania/NOAA_NDVI_Forecasting_IDL/blob/main/Midterm_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Minimum code that lets you download a single file - should be enough to start parsing the basic data format.

In [ ]:
### MINIMUM DOWNLOAD FOR SINGLE FILE ###
# can change that url to any file url from https://noaa-cdr-hydrological-properties-pds.s3.amazonaws.com/index.html
!aws s3 cp s3://noaa-cdr-hydrological-properties-pds/data/ . --no-sign-request --region us-east-1

import xarray as xr

ds = xr.open_dataset("AOT_AVHRR_v04r00-preliminary_monthly-avg_202301_c20230418.nc")
print(ds)

download: s3://noaa-cdr-aerosol-optical-thickness-pds/data/monthly/AOT_AVHRR_v04r00-preliminary_monthly-avg_202301_c20230418.nc to ./AOT_AVHRR_v04r00-preliminary_monthly-avg_202301_c20230418.nc
<xarray.Dataset> Size: 26MB
Dimensions:      (time: 1, latitude: 1800, longitude: 3600, nv: 2)
Coordinates:
  * time         (time) datetime64[ns] 8B 2023-01-01
  * latitude     (latitude) float32 7kB -89.95 -89.85 -89.75 ... 89.85 89.95
  * longitude    (longitude) float32 14kB -179.9 -179.8 -179.8 ... 179.8 179.9
Dimensions without coordinates: nv
Data variables:
    aot1         (time, latitude, longitude) float32 26MB ...
    lat_bounds   (latitude, nv) float32 14kB ...
    lon_bounds   (longitude, nv) float32 29kB ...
    time_bounds  (time, nv) datetime64[ns] 16B ...
Attributes: (12/46)
    Conventions:                CF-1.6, ACDD-1.3
    title:                      Monthly AVHRR Aerosol Optical Thickness over ...
    references:                 doi: 10.1002/jgrd.50278.2015 and AVHRR_AOT_C

In [ ]:
# double check that we can access the datasource + we have the valid packages to access the data
!apt remove awscli -y
!pip install boto3 s3fs xarray netCDF4 zarr
!pip install awscli --upgrade
!aws s3 ls s3://noaa-cdr-hydrological-properties-pds/ --region us-east-1 --no-sign-request | head
!aws s3 ls s3://noaa-cdr-ndvi-pds/ --region us-east-1 --no-sign-request | head


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package 'awscli' is not installed, so not removed
The following packages were automatically installed and are no longer required:
  docutils-common fonts-droid-fallback fonts-noto-mono ghostscript groff
  gsfonts imagemagick imagemagick-6-common imagemagick-6.q16 libdjvulibre-text
  libdjvulibre21 libfftw3-double3 libgs9 libgs9-common libidn12 libijs-0.35
  libimagequant0 libjbig2dec0 libjxr-tools libjxr0 liblqr-1-0
  libmagickcore-6.q16-6 libmagickcore-6.q16-6-extra libmagickwand-6.q16-6
  libnetpbm10 libraqm0 libwmflite-0.2-7 netpbm psutils python3-botocore
  python3-certifi python3-chardet python3-colorama python3-dateutil
  python3-docutils python3-idna python3-jmespath python3-olefile python3-pil
  python3-pyasn1 python3-requests python3-roman python3-rsa python3-s3transfer
  python3-urllib3 sgml-base xml-core
Use 'apt autoremove' to remove them.
0 upgraded, 0 newly installed, 0 to rem

In [ ]:
# CODE SCAFFOLDING:

"""
overall setup for code:
DATA PROCESSING:
1. load the data from raw noaa files into large array of chronologically sorted vectors (date, all other features in a F X N array) (save as a csv if its not too massive???)
  - first index should store week data (i think counting up from 0 --> the last week number woul dmake sense)
2. collcet the ndvi data from the same timeframe and append that data to our massive array
3. convert entire training set to train/ validation/ test (first 75% is train, next 10% is validation, last 15% is train; chronologically speaking)
  some people on the internet say the validation and test splits should overlap slightly, but i havent validated yet.
4. batch the training set and store those batches in a dataloader for training purposes.


MODEL TRAINING:
1. write an LSTM class
2.
"""

In [ ]:
import os
import re
import datetime
import subprocess

def s3_file_exists(s3_path):
      """Check whether an object exists in the S3 bucket (no sign-in required)."""
      cmd = ["aws", "s3", "ls", s3_path, "--no-sign-request", "--region", region]
      result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
      return result.returncode == 0 and len(result.stdout) > 0


def download_noaa_hydrological(start_year, end_year, step_days=7):
    bucket = "noaa-cdr-hydrological-properties-pds"
    prefix = "data"
    region = "us-east-1"

    base_dir = "data/hydrological"
    os.makedirs(base_dir, exist_ok=True)

    doy_pattern = re.compile(r"D(\d{2})(\d{3})")  # matches DYYDDD, e.g. D17121

    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            month_str = f"{month:02d}"
            month_dir = os.path.join(base_dir, f"{year}_{month_str}")
            os.makedirs(month_dir, exist_ok=True)

            print(f"\n📅 Listing Hydrological files for {year}-{month_str}...")
            cmd = [
                "aws", "s3", "ls", f"s3://{bucket}/{prefix}/{year}/{month_str}/",
                "--no-sign-request", "--region", region
            ]
            result = subprocess.run(cmd, capture_output=True, text=True)
            if result.returncode != 0 or not result.stdout.strip():
                print(f"⚠️ No data for {year}-{month_str}")
                continue

            # Extract filenames
            files = [line.split()[-1] for line in result.stdout.splitlines() if line.endswith(".nc")]

            # Parse approximate date from filename
            def extract_date(fname):
                m = doy_pattern.search(fname)
                if not m:
                    return None
                yy, doy = int(m.group(1)), int(m.group(2))
                year_full = 2000 + yy if yy < 50 else 1900 + yy  # crude century logic
                return datetime.datetime(year_full, 1, 1) + datetime.timedelta(days=doy - 1)

            file_dates = [(f, extract_date(f)) for f in files]
            file_dates = [(f, d) for f, d in file_dates if d]
            file_dates.sort(key=lambda x: x[1])

            if not file_dates:
                print(f"⚠️ Could not extract dates for {year}-{month_str}")
                continue

            # Sample roughly one per step_days
            sampled = []
            last_date = None
            for fname, date in file_dates:
                if not last_date or (date - last_date).days >= step_days:
                    sampled.append(fname)
                    last_date = date

            print(f"Found {len(files)} files, sampled {len(sampled)} (~1 per {step_days} days)")

            for fname in sampled:
                s3_path = f"s3://{bucket}/{prefix}/{year}/{month_str}/{fname}"
                local_path = os.path.join(month_dir, fname)

                if os.path.exists(local_path):
                    print(f"✅ Already downloaded: {fname}")
                    continue

                print(f"⬇️ Downloading: {fname}")
                subprocess.run([
                    "aws", "s3", "cp", s3_path, local_path,
                    "--no-sign-request", "--region", region
                ])

    print("🎉 Hydrological download complete!")


def download_ndvi_files(start_year, end_year, step_days):
    bucket = "noaa-cdr-ndvi-pds"
    prefix = "data"
    region = "us-east-1"

    base_dir = "data"
    os.makedirs(base_dir, exist_ok=True)

    for year in range(start_year, end_year + 1):
        year_dir = os.path.join(base_dir, str(year))
        os.makedirs(year_dir, exist_ok=True)

        print(f"\n📅 Listing files for year {year}...")
        cmd = [
            "aws", "s3", "ls", f"s3://{bucket}/{prefix}/{year}/",
            "--no-sign-request", "--region", region
        ]
        result = subprocess.run(cmd, capture_output=True, text=True)

        if result.returncode != 0:
            print(f"⚠️  Could not list files for {year}. Skipping.")
            continue

        lines = result.stdout.strip().split("\n")
        files = [line.split()[-1] for line in lines if line.strip().endswith(".nc")]

        if not files:
            print(f"⚠️  No .nc files found for {year}")
            continue

        print(f"Found {len(files)} files for {year}")

        # Sort chronologically (they usually include YYYYMMDD in filename)
        files.sort()

        for i, fname in enumerate(files[::step_days]):  # sample every Nth file
            s3_path = f"s3://{bucket}/{prefix}/{year}/{fname}"
            local_path = os.path.join(year_dir, fname)

            if os.path.exists(local_path):
                print(f"✅ Already downloaded: {fname}")
                continue

            print(f"⬇️  Downloading: {fname}")
            subprocess.run([
                "aws", "s3", "cp", s3_path, local_path,
                "--no-sign-request", "--region", region
            ])

    print("🎉 Done! All available NDVI files downloaded.")



# loads data and stores data features
def load_raw_noaa_data() -> torch.Tensor:
  download_hydrological_files(start_year, end_year)
  download_ndvi_files(start_year, end_year)
  # process the files into a tensor: not sure what the raw format of each file is.
  # TODO: download one year's worth of files ot determine basic format and convert to tensor.




# TODO: DECIDE IF DATALOADERS ARE A GOOD IDEA BASED ON WHAT TORCH.NN.LSTM TAKES IN
def create_dataloaders(raw_tensor, batch_size=32) -> (torch.DataLoader, torch.DataLoader, torch.DataSet):
  pass




In [ ]:
# defines the model

import torch.nn as nn

class BasicLSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=2, lr=0.001):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

        self.model = nn.Sequential((self.lstm, self.fc))
        self.loss_fn = nn.MSELoss()
        self.optim = torch.optim.AdamW(self.model.parameters(), lr=lr)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])
        return out

    def train(train_loader, epochs=10):
      losses = []
      for epoch in epochs:
        running_loss = 0
        for i, data in enumerate(train_loader):
          inputs, labels = data
          self.optim.zero_grad()

          preds = self.model(inputs)
          loss = self.loss_fn(pred, labels)
          loss.backward()

          self.optim.step()
          running_loss += loss.item()
        losses.append(running_loss)
      return losses

def test(test_data):
  pass


In [ ]:
# actually call all the code!
# HYPERPARAMETERS (to tune)
batch_size = 32
hidden_size = 64 # idk
lr = 0.001
epoch_count = 10

raw_tensor = load_raw_noaa_data()
train_loader, val_loader, test_set = create_dataloaders(raw_tensor, batch_size=32)

# DATA FEATURES
vector_size, num_weeks = raw_tensor.shape()
input_size = vector_size - 1

model = BasicLSTM(input_size=input_size, hidden_size=hidden_size)
model.train(train_loader)
model.test(test_set)